# What-If Tool Challenge Lab

In this notebook, you will use mortgage data from NY in 2017 to create two binary classifiers to determine if a mortgage applicant will be granted a loan.

You will train classifiers on two datasets. One will be trained on the complete dataset, and the other will be trained on a subset of the dataset, where 90% of the female applicants that were granted a loan were removed from the training data (so the dataset has 90% less females that were granted loans).

You will then compare and examine the two models using the What-If Tool.

In this notebook, you will be exepcted to:
* Understand how the data is processed 
* Write TensorFlow code to build and train two models
* Write code to deploy the the models to AI Platform
* Examine the models in the What-If Tool

# Download and import the data

Here, you'll import some modules and download some data from the Consumer Finance public [datasets](https://www.consumerfinance.gov/data-research/hmda/historic-data/?geo=ny&records=all-records&field_descriptions=labels).

In [1]:
import pandas as pd
import numpy as np
import collections
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
!wget https://files.consumerfinance.gov/hmda-historic-loan-data/hmda_2017_ny_all-records_labels.zip
!unzip hmda_2017_ny_all-records_labels.zip

--2021-10-06 06:15:31--  https://files.consumerfinance.gov/hmda-historic-loan-data/hmda_2017_ny_all-records_labels.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 52.84.162.103, 52.84.162.24, 52.84.162.10, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|52.84.162.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17466285 (17M) [application/zip]
Saving to: ‘hmda_2017_ny_all-records_labels.zip’

hmda_2017_ny_all-re 100%[===================>]  16.66M  --.-KB/s    in 0.09s   

2021-10-06 06:15:31 (181 MB/s) - ‘hmda_2017_ny_all-records_labels.zip’ saved [17466285/17466285]

Archive:  hmda_2017_ny_all-records_labels.zip
  inflating: hmda_2017_ny_all-records_labels.csv  


# Process the Data

In this section, you **don't need to write any code**. We suggest you read through the cells to understand how the dataset is processed.

Here, we start by importing the dataset into a Pandas dataframe. Then we process the data to exclude incomplete information and make a simple binary classification of loan approvals. We then create two datasets, one complete and one where 90% of female applicants are removed.

In [3]:
# Set column dtypes for Pandas
column_names = collections.OrderedDict({
  'as_of_year': np.int16,
  'agency_abbr': 'category',
  'loan_type': 'category',
  'property_type': 'category',
  'loan_purpose': 'category',
  'owner_occupancy': np.int8,
  'loan_amt_000s': np.float64,
  'preapproval': 'category',
  'county_code': np.float64,
  'applicant_income_00s': np.float64,
  'purchaser_type': 'category',
  'hoepa_status': 'category',
  'lien_status': 'category',
  'population': np.float64,
  'ffiec_median_fam_income': np.float64,
  'tract_to_msamd_income': np.float64,
  'num_of_owner_occupied_units': np.float64,
  'number_of_1_to_4_family_units': np.float64,
  'approved': np.int8, 
  'applicant_race_name_3': 'category',
  'applicant_race_name_4': 'category',
  'applicant_race_name_5': 'category',
  'co_applicant_race_name_3': 'category',
  'co_applicant_race_name_4': 'category',
  'co_applicant_race_name_5': 'category'
})

# Import the CSV into a dataframe
data = pd.read_csv('hmda_2017_ny_all-records_labels.csv', dtype=column_names)
data = shuffle(data, random_state=2)

## Extract columns and create dummy dataframes

We first specify which columns to keep then drop the columns that don't have `loan originated` or `loan denied`, to make this a simple binary classification.

We then create two dataframes `binary_df` and `bad_binary_df`. The first will include all the data, and the second will have 90% of female applicants removed, respectively. We then convert them into "dummy" dataframes to turn categorical string features into simple 0/1 features and normalize all the columns.

In [4]:
# Only use a subset of the columns for these models
text_columns_to_keep = [
             'agency_name',
             'loan_type_name',
             'property_type_name',
             'loan_purpose_name',
             'owner_occupancy_name',
             'applicant_ethnicity_name',
             'applicant_race_name_1',
             'applicant_sex_name',                      
]
numeric_columns_to_keep = [
             'loan_amount_000s',
             'applicant_income_000s',
             'population',
             'minority_population',
             'hud_median_family_income' 
]

columns_to_keep = text_columns_to_keep + numeric_columns_to_keep + ['action_taken_name']

# Drop columns with incomplete information and drop columns that don't have loan orignated or denied, to make this a simple binary classification
df = data[columns_to_keep].dropna()
binary_df = df[df.action_taken_name.isin(['Loan originated', 'Application denied by financial institution'])].copy()
binary_df.loc[:,'loan_granted'] = np.where(binary_df['action_taken_name'] == 'Loan originated', 1, 0)
binary_df = binary_df.drop(columns=['action_taken_name'])

# Drop 90% of loaned female applicants for a "bad training data" version
loaned_females = (binary_df['applicant_sex_name'] == 'Female') & (binary_df['loan_granted'] == 1)
bad_binary_df = binary_df.drop(binary_df[loaned_females].sample(frac=.9).index)

In [5]:
# Now lets' see the distribution of approved / denied classes (0: denied, 1: approved)
print(binary_df['loan_granted'].value_counts())

1    223026
0     63001
Name: loan_granted, dtype: int64


In [6]:
# Turn categorical string features into simple 0/1 features (like turning "sex" into "sex_male" and "sex_female")
dummies_df = pd.get_dummies(binary_df, columns=text_columns_to_keep)
dummies_df = dummies_df.sample(frac=1).reset_index(drop=True)

bad_dummies_df = pd.get_dummies(bad_binary_df, columns=text_columns_to_keep)
bad_dummies_df = bad_dummies_df.sample(frac=1).reset_index(drop=True)

In [7]:
# Normalize the numeric columns so that they all have the same scale to simplify modeling/training
def normalize():
  min_max_scaler = preprocessing.MinMaxScaler()
  column_names_to_normalize = ['loan_amount_000s', 'applicant_income_000s', 'minority_population', 'hud_median_family_income', 'population']
  x = dummies_df[column_names_to_normalize].values
  x_scaled = min_max_scaler.fit_transform(x)
  df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = dummies_df.index)
  dummies_df[column_names_to_normalize] = df_temp

  x = bad_dummies_df[column_names_to_normalize].values
  x_scaled = min_max_scaler.fit_transform(x)
  bad_df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = bad_dummies_df.index)
  bad_dummies_df[column_names_to_normalize] = bad_df_temp

normalize()

## Get the Train & Test Data

Now, let's get the train and test data for our models.

For the **first** model, you'll use `train_data` and `train_labels`.

For the **second** model, you'll use `limited_train_data` and `limited_train_labels`.

In [8]:
# Get the training data & labels
test_data_with_labels = dummies_df

train_data = dummies_df
train_labels = train_data['loan_granted']
train_data = train_data.drop(columns=['loan_granted'])

# Get the bad (limited) training data and labels
limited_train_data = bad_dummies_df
limited_train_labels = limited_train_data['loan_granted']
limited_train_data = bad_dummies_df.drop(columns=['loan_granted'])

# Split the data into train / test sets for Model 1
x,y = train_data,train_labels
train_data,test_data,train_labels,test_labels = train_test_split(x,y)

# Split the bad data into train / test sets for Model 2
lim_x,lim_y=limited_train_data,limited_train_labels
limited_train_data,limited_test_data,limited_train_labels,limited_test_labels = train_test_split(lim_x,lim_y)

# Create and train your TensorFlow models

In this section, you will write code to train two TensorFlow Keras models.

## Train your first model on the complete dataset.

* **Important**: your first model should be saved in the location `saved_complete_model/saved_model.pb`.
* The data will come from `train_data` and `train_labels`.

If you get stuck, you can view the documentation [here](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential).

In [9]:
# import TF modules
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
# This is the size of the array you'll be feeding into our model for each example
input_size = len(train_data.iloc[0])

# Train the first model on the complete dataset. Use `train_data` for your data and `train_labels` for you labels.
model = Sequential()
model.add(layers.Dense(8, input_dim=input_size))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='sgd', loss='mse')
model.fit(train_data, train_labels, batch_size=32, epochs=10)
# ---- TODO ---------
# create the model = Sequential()
# model.add (your layers)
# model.compile
# model.fit

2021-10-06 06:16:29.178029: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200215000 Hz
2021-10-06 06:16:29.178908: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557e3ee0c640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-06 06:16:29.178945: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-06 06:16:29.181269: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
6704/6704 [==============================] - 13s 2ms/step - loss: 0.1628
Epoch 2/10
6704/6704 [==============================] - 13s 2ms/step - loss: 0.1561
Epoch 3/10
6704/6704 [==============================] - 12s 2ms/step - loss: 0.1550
Epoch 4/10
6704/6704 [==============================] - 17s 3ms/step - loss: 0.1545
Epoch 5/10
6704/6704 [==============================] - 13s 2ms/step - loss: 0.1543
Epoch 6/10
6704/6704 [==============================] - 13s 2ms/step - loss: 0.1541
Epoch 7/10
6704/6704 [==============================] - 12s 2ms/step - loss: 0.1540
Epoch 8/10
6704/6704 [==============================] - 13s 2ms/step - loss: 0.1540
Epoch 9/10
6704/6704 [==============================] - 14s 2ms/step - loss: 0.1539
Epoch 10/10
6704/6704 [==============================] - 12s 2ms/step - loss: 0.1539


In [11]:
# Save your model
!mkdir -p saved_complete_model
model.save('saved_complete_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-10-06 06:18:47.455466: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_complete_model/assets


In [12]:
# Get predictions on the test set and print the accuracy score (Model 1)
y_pred = model.predict(test_data)
acc = accuracy_score(test_labels, y_pred.round())
print("Model 1 Accuracy: %.2f%%" % (acc * 100.0))

Model 1 Accuracy: 78.27%


## Train your second model on the limited datset.

* **Important**: your second model should be saved in the location `saved_limited_model/saved_model.pb`.
* The data will come from `limited_train_data` and `limited_train_labels`.


If you get stuck, you can view the documentation [here](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential).

In [13]:
# Train your second model on the limited dataset. Use `limited_train_data` for your data and `limited_train_labels` for your labels.
# Use the same input_size for the limited_model
limited_model = Sequential()
limited_model.add(layers.Dense(8, input_dim=input_size))
limited_model.add(layers.Dense(1, activation='sigmoid'))
limited_model.compile(optimizer='sgd', loss='mse')
limited_model.fit(limited_train_data, limited_train_labels, batch_size=32, epochs=10)

# ---- TODO ---------
# create the limited_model = Sequential()
# limited_model.add (your layers)
# limited_model.compile
# limited_model.fit

Epoch 1/10
5215/5215 [==============================] - 10s 2ms/step - loss: 0.1654
Epoch 2/10
5215/5215 [==============================] - 12s 2ms/step - loss: 0.1557
Epoch 3/10
5215/5215 [==============================] - 11s 2ms/step - loss: 0.1551
Epoch 4/10
5215/5215 [==============================] - 10s 2ms/step - loss: 0.1550
Epoch 5/10
5215/5215 [==============================] - 11s 2ms/step - loss: 0.1549
Epoch 6/10
5215/5215 [==============================] - 10s 2ms/step - loss: 0.1549
Epoch 7/10
5215/5215 [==============================] - 11s 2ms/step - loss: 0.1549
Epoch 8/10
5215/5215 [==============================] - 10s 2ms/step - loss: 0.1549
Epoch 9/10
5215/5215 [==============================] - 9s 2ms/step - loss: 0.1549
Epoch 10/10
5215/5215 [==============================] - 9s 2ms/step - loss: 0.1548


In [14]:
# Save your model
!mkdir -p saved_limited_model
limited_model.save('saved_limited_model') 

INFO:tensorflow:Assets written to: saved_limited_model/assets


In [15]:
# Get predictions on the test set and print the accuracy score (Model 2)
limited_y_pred = limited_model.predict(limited_test_data)
acc = accuracy_score(limited_test_labels, limited_y_pred.round())
print("Model 2 Accuracy: %.2f%%" % (acc * 100.0))

Model 2 Accuracy: 77.97%


# Deploy your models to the AI Platform

In this section, you will first need to create a Cloud Storage bucket to store your models, then you will use gcloud commands to copy them over.

You will then create two AI Platform model resources and their associated versions.

In [46]:
# ---- TODO ---------

# Fill out this information:

GCP_PROJECT = 'qwiklabs-gcp-04-0c91a78cd9d6'
MODEL_BUCKET = 'gs://qwiklabs-gcp-04-0c91a78cd9d6'
MODEL_NAME = 'saved_complete_model' #do not modify
LIM_MODEL_NAME = 'saved_limited_model' #do not modify
VERSION_NAME = 'v1'
REGION = 'us-central1'

In [47]:
# Copy your model files to Cloud Storage (these file paths are your 'origin' for the AI Platform Model)
!gsutil cp -r ./saved_complete_model $MODEL_BUCKET
!gsutil cp -r ./saved_limited_model $MODEL_BUCKET

Copying file://./saved_complete_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./saved_complete_model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./saved_complete_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
- [3 files][ 54.8 KiB/ 54.8 KiB]                                                
Operation completed over 3 objects/54.8 KiB.                                     
Copying file://./saved_limited_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./saved_limited_model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./saved_limited_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
- [3 files][ 55.4 KiB/ 55.4 KiB]                                                
Operation completed over 3 objects/55.4 KiB.                                     


In [48]:
# Configure gcloud to use your project
!gcloud config set project $GCP_PROJECT
!gcloud config set ai_platform/region global

Updated property [core/project].
Updated property [ai_platform/region].


## Create your first AI Platform model: *saved_complete_model*

Navigate back to the Google Cloud Console to complete this task. See the lab guide for details.

In [61]:
!gcloud ai-platform models create complete_model --regions $REGION

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/qwiklabs-gcp-04-0c91a78cd9d6/models/complete_model].


In [ ]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=complete_model \
--framework='TENSORFLOW' \
--runtime-version=2.1 \
--origin=$MODEL_BUCKET/saved_complete_model/ \
--staging-bucket=$MODEL_BUCKET \
--python-version=3.7

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......⠧                        

## Create your second AI Platform model: *saved_limited_model*

Navigate back to the Google Cloud Console to complete this task. See the lab guide for details.

In [59]:
!gcloud ai-platform models create limited_model --regions $REGION

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/qwiklabs-gcp-04-0c91a78cd9d6/models/limited_model].


In [60]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=limited_model \
--framework='TENSORFLOW' \
--runtime-version=2.3 \
--origin=$MODEL_BUCKET/saved_limited_model \
--staging-bucket=$MODEL_BUCKET \
--python-version=3.7

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


# Using the What-if Tool to interpret your model
Once your models have deployed, you're now ready to connect them to the What-if Tool using the WitWidget. 

We've provided the Config Builder code and a couple of functions to get the class predictions from the models, which are necessary inputs for the WIT. If you've successfully deployed and saved your models, all you'll need to do is **add the WitConfigBuilder code in this cell**.

In [57]:
#@title Show model results in WIT
num_datapoints = 1000  #@param {type: "number"}

# Column indices to strip out from data from WIT before passing it to the model.
columns_not_for_model_input = [
    test_data_with_labels.columns.get_loc('loan_granted'),
]

# Return model predictions.
def custom_predict(examples_to_infer):
  # Delete columns not used by model
  model_inputs = np.delete(
      np.array(examples_to_infer), columns_not_for_model_input, axis=1).tolist()
  # Get the class predictions from the model.
  preds = model.predict(model_inputs)
  preds = [[1 - pred[0], pred[0]] for pred in preds]
  return preds
  
# Return 'limited' model predictions.
def limited_custom_predict(examples_to_infer):
  # Delete columns not used by model
  model_inputs = np.delete(
      np.array(examples_to_infer), columns_not_for_model_input, axis=1).tolist()
  # Get the class predictions from the model.
  preds = limited_model.predict(model_inputs)
  preds = [[1 - pred[0], pred[0]] for pred in preds]
  return preds

examples_for_wit = test_data_with_labels.values.tolist()
column_names = test_data_with_labels.columns.tolist()

config_builder = (WitConfigBuilder(
    examples_for_wit[:num_datapoints],feature_names=column_names)
    .set_custom_predict_fn(limited_custom_predict)
    .set_target_feature('loan_granted')
    .set_label_vocab(['denied', 'accepted'])
    .set_compare_custom_predict_fn(custom_predict)
    .set_model_name('limited')
    .set_compare_model_name('complete'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'classification', 'label_vocab': ['denied', 'accepted'], 'feature_names': ['lo…